# Heirarchical Cluster probability models

The question is given a heirarchical clustering model what is the actual probability distribution on inputs?

Here I want to explore how to calculate the actual input probability distribution.

In [2]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Normal, MultivariateNormal, Categorical
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import datasets, transforms
from torch.nn.utils.clip_grad import clip_grad_value_, clip_grad_norm_

import string
from random import *
import re
import math
import time

import subprocess, os, glob
import time

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

device = 'cuda:2'
print (device)

print (os.getcwd())

print(torch.__version__)

from dennytools.silence import *
from Modules.conv import Convolutor
from Distributions import MultinouliClassification

cuda:2
/home/tbjackso/NeoNeo
1.2.0


In [3]:

class EMMB(nn.Module):
    def __init__(self, D, M, eta=0.95, FF=None):
        super().__init__()
        self.register_buffer('D', torch.tensor(D))
        self.register_buffer('M', torch.tensor(M))
        
        self.pi = nn.Parameter(torch.ones((1,M))/M, requires_grad=False)
        self.mu = nn.Parameter(torch.rand((1,M,D)), requires_grad=False)
        
        self.FF = nn.Softmax(dim=-1) if FF is None else FF
        
    def responsibility(self, x, raw=False, verbose=False):
        with silence(not verbose):
            print ('x', torch.isnan(x).sum()>0)
            print ('pi', torch.isnan(self.pi).sum()>0)
            print ('mu', torch.isnan(self.mu).sum()>0)
            # Negative Log Liklihood of the multinoulis
            logpi = torch.log(self.pi+1e-18)
            print ('logpi',torch.isnan(logpi).sum()>0)
            logmu = torch.log(self.mu+1e-18)
            print ('logmu',torch.isnan(logmu).sum()>0)
            logommu = torch.log(1-self.mu+1e-18)
            print ('logommu',torch.isnan(logommu).sum()>0)
            mix_log_probs = (logpi + (x*logmu + (1-x)*logommu).sum(-1))
            print ('mix_log_probs', torch.isnan(mix_log_probs).sum()>0)
            resp = F.softmax(mix_log_probs, dim=-1)
            print ('resp', torch.isnan(resp).sum()>0)

        if (raw):
            return mix_log_probs
        return resp
        
    def forward(self, x):
        x = x.view(-1,1,self.D)
        y = self.responsibility(x, raw=True)
        return self.FF(y)
        
    def learn(self, x):
        x = x.view(-1,1,self.D)
        
        # This learns via EM algorithm
        # E step: calculates the responsibility of each mode for each point
        # M step: calculates the pis and mus
        rik = self.responsibility(x)
        if (torch.isnan(rik).sum()>0):
            self.responsibility(x, verbose=True)
            raise ValueError("NaN Found")
        rk = rik.sum(0, keepdim=True)
        # rik:  (BS, M) -> (BS, M, 1)
        newmu = (rik.unsqueeze(-1)*x).sum(0, keepdim=True)/(rk.unsqueeze(-1)+1e-18)
        newpi = rk/x.shape[0]
        
        # The M update
        self.pi.data = eta*self.pi.data + (1-eta)*newpi
        self.mu.data = eta*self.mu.data + (1-eta)*newmu
        
    def params(self, sort=True):
        pi = self.pi.squeeze()
        mu = self.mu.squeeze()
        if (sort):
            pi, pin = pi.sort(descending=True)
            mu = mu[pin]
        return (pi, mu)

In [17]:
BS = 100

C = 2
M = 1
D = 4

MMDataset = MultinouliClassification(C, M, D, noise=0.1, samples=5000)
# Training dataset
train = torch.utils.data.DataLoader(MMDataset, batch_size=BS, shuffle=True)

print (len(train))
print (len(MMDataset))

100
10000


In [18]:
(data, label) = next(iter(train))

In [19]:
print (data[:5])
print (MMDataset.archetypes)

archetypes = MMDataset.archetypes
pi = torch.ones((1,C*M))/C*M

# For fun... lets see the likelihood that a given data sample belongs to a given 
# archetype (Ideally these archetypes will be learned by the clustering method)

def responsibility(x, pi, mu, raw=False, verbose=False):
    with silence(not verbose):
        x = x.unsqueeze(1)
        print ('x', torch.isnan(x).sum()>0)
        print ('pi', torch.isnan(pi).sum()>0)
        print ('mu', torch.isnan(mu).sum()>0)
        # Negative Log Liklihood of the multinoulis
        logpi = torch.log(pi+1e-18)
        print ('logpi',torch.isnan(logpi).sum()>0)
        logmu = torch.log(mu+1e-18)
        print ('logmu',torch.isnan(logmu).sum()>0)
        logommu = torch.log(1-mu+1e-18)
        print ('logommu',torch.isnan(logommu).sum()>0)
        mix_log_probs = (logpi + (x*logmu + (1-x)*logommu).sum(-1))
        print ('mix_log_probs', torch.isnan(mix_log_probs).sum()>0)
        resp = F.softmax(mix_log_probs, dim=-1)
        print ('resp', torch.isnan(resp).sum()>0)
    
    if (raw):
        return mix_log_probs
    return resp

resp = responsibility(data, pi, archetypes)
print (resp[:5])

MLP = responsibility(data, pi, archetypes, raw=True)
print (MLP.shape)
print (MLP[0])
print (data[0])
print (archetypes[-1])
TP = 0
for i in range(C):
    p = pi[0,i]
    for j in range(D):
        p *= (data[0,j]*archetypes[i,j] + (1-data[0,j])*(1-archetypes[i,j]))
    print (np.log(p))
    TP += p
    
print (np.log(TP))
# Checks out lets do this thing

tensor([[1., 0., 0., 0.],
        [1., 1., 1., 1.],
        [0., 0., 0., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])
tensor([[0.9000, 0.1000, 0.1000, 0.1000],
        [0.9000, 0.9000, 0.9000, 0.9000]])
tensor([[0.9986, 0.0014],
        [0.0014, 0.9986],
        [0.9000, 0.1000],
        [0.0014, 0.9986],
        [0.0014, 0.9986]])
torch.Size([100, 2])
tensor([-1.1146, -7.7063])
tensor([1., 0., 0., 0.])
tensor([0.9000, 0.9000, 0.9000, 0.9000])
tensor(-1.1146)
tensor(-7.7063)
tensor(-1.1132)


In [ ]:
p = 0.9**9 * 0.1
print (p)
print (np.log(p))

In [20]:
# Lets make a model and do that training thing

eta = 0.95
epochs = 5

model = EMMB(D, C*M, eta=eta)
# model.mu.data = archetypes.unsqueeze(0)

for e in range(epochs):
    print ("Epoch: {}".format(e))
    for (data, label) in train:
        model.learn(data)
        
(pi, mu) = model.params()

print (pi)
mask = (mu > 1e-30).float()
print (mu*mask)
print (archetypes)
print (pi)
        

Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
tensor([0.5139, 0.4861])
tensor([[0.9037, 0.9094, 0.9121, 0.9037],
        [0.9127, 0.1121, 0.0939, 0.1037]])
tensor([[0.9000, 0.1000, 0.1000, 0.1000],
        [0.9000, 0.9000, 0.9000, 0.9000]])
tensor([0.5139, 0.4861])


In [23]:
(pi,mu) = model.params()
print (pi)
print (mu)
print (mu.shape)
archetypes = archetypes
resp = responsibility(archetypes, pi.unsqueeze(0), mu.unsqueeze(0))
mask = (resp>1e-10).float()
print (pi)
print (resp*mask)
print ("\n-----------\n")
print (mu[0])
print (archetypes[1])
print ("\n-------------")
data = (archetypes>0.5).float()
print (data)
print ("-------------\n")

# raise ValueError('STOP')

ones = data @ data.transpose(0,1)
zeros = (1-data) @ (1-data).transpose(0,1)
overlap = ones + zeros
dist = 10 - overlap
print(overlap)
print (dist)

print (0.9**overlap)
P = (0.9**overlap + 0.1**dist)
print (P)
print (P.mean(-1))
print (torch.log(P.mean(-1)))

tensor([0.5139, 0.4861])
tensor([[0.9037, 0.9094, 0.9121, 0.9037],
        [0.9127, 0.1121, 0.0939, 0.1037]])
torch.Size([2, 4])
tensor([0.5139, 0.4861])
tensor([[0.0043, 0.9957],
        [0.9948, 0.0052]])

-----------

tensor([0.9037, 0.9094, 0.9121, 0.9037])
tensor([0.9000, 0.9000, 0.9000, 0.9000])

-------------
tensor([[1., 0., 0., 0.],
        [1., 1., 1., 1.]])
-------------

tensor([[4., 1.],
        [1., 4.]])
tensor([[6., 9.],
        [9., 6.]])
tensor([[0.6561, 0.9000],
        [0.9000, 0.6561]])
tensor([[0.6561, 0.9000],
        [0.9000, 0.6561]])
tensor([0.7781, 0.7781])
tensor([-0.2510, -0.2510])


In [ ]:
data = (archetypes>0.5).float()
print (archetypes.shape)
print (responsibility(data, torch.ones(1,C)/C, archetypes.unsqueeze(0)))

In [8]:
# This cell demonstrates how to do offline EM clustering
# It it kept solely for historical reference

epochs = 10

# mu = torch.rand((1,C*M,D))
mu = archetypes.unsqueeze(0)
pi = torch.ones((1,C*M))/(C*M)

print (pi)
print (mu)

# E step: calculates the responsibility of each mode for each point
# M step: calculates the pis and mus

# Since we are doing this with batches we need to accumulate the statistics
for e in range(epochs):
    print ('Epoch: {}'.format(e))
    newmu = torch.zeros_like(mu)
    rk = torch.zeros_like(pi)
    N = 0
    for (data, label) in train:
        data = data.view(-1,D)
        N += data.shape[0]
        rik = responsibility(data, pi, mu)
        if (torch.isnan(rik).sum()>0):
            responsibility(data, pi, mu, verbose=True)
            raise ValueError("NaN Found")
        rk += rik.sum(0, keepdim=True)
        # rik:  (BS, M) -> (BS, M, 1)
        # data: (BS, D) -> (BS, 1, D)
        newmu += (rik.unsqueeze(-1)*data.unsqueeze(1)).sum(0, keepdim=True)
        
    pi = rk/N
    mu = newmu/rk.unsqueeze(-1)

print (pi)
print (mu)


tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000]])
tensor([[[0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.1000, 0.1000],
         [0.9000, 0.1000, 0.9000,  ..., 0.1000, 0.9000, 0.9000],
         [0.1000, 0.9000, 0.1000,  ..., 0.9000, 0.9000, 0.1000],
         ...,
         [0.1000, 0.1000, 0.9000,  ..., 0.9000, 0.1000, 0.9000],
         [0.1000, 0.1000, 0.1000,  ..., 0.1000, 0.9000, 0.9000],
         [0.1000, 0.1000, 0.1000,  ..., 0.9000, 0.9000, 0.1000]]])
Epoch: 0
Epoch: 1
Epoch: 2
Epoch: 3
Epoch: 4
Epoch: 5
Epoch: 6
Epoch: 7
Epoch: 8
Epoch: 9
tensor([[0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000, 0.1000,
         0.1000]])
tensor([[[0.1020, 0.1060, 0.0962,  ..., 0.1040, 0.1076, 0.1060],
         [0.9046, 0.1026, 0.9008,  ..., 0.1022, 0.9024, 0.8968],
         [0.0970, 0.9018, 0.1058,  ..., 0.8938, 0.9032, 0.1042],
         ...,
         [0.1046, 0.0976, 0.9084,  ..., 0.9020, 0.0922, 0.9040],
         [0.1018, 0.1014, 0.10

In [ ]:
#
binary = []
for i in range(16):
    binary.append(np.unpackbits(np.array([i], dtype=np.uint8)))
binary = np.array(binary)
print (binary)

D = 4
P = np.zeros([2]*D)
print (P.shape)

P[0,0,0,0] = 1
P[1,1,1,1] = 1
P[0,0,0,1] = 0
P[1,1,1,0] = 0

P = P/P.sum()
print (P)

In [ ]:
import numpy as np
# Helper function
def str2bool(v):
    return 0 if v.lower() in ("yes", "true", "t", "1") else 1

class factor():
    def __init__(self, array, variables):
        # None factor - The empty Factor allows me to avoid being concerned
        #about edge cases user side
        if (len(variables) == 0):
            self.data = np.array([])
            self.vars = []
            self.isEmpty = True
            return
        
        # Construct the data from an array passed in
        data = np.array(array)
        # Check that the correct number of dimension labels is passed in
        if (len(variables) != data.ndim):
            raise ValueError("Number of labels does not match number of dimensions")
        
        self.data = data
        self.vars = variables
        self.isEmpty = False

    # Implementation of the factor restrict function
    def restrict(self, var, value):
        # If empty factor return an new empty factor
        if self.isEmpty:
            return factor([],[])
        
        # This essentially helps translate human readable values to internal numbers
        # note that internally a 0 value is the T value for that dimension which 
        # is a little counter intuitive hence bothering with a conversion
        if isinstance(value, str):
            value = str2bool(value)
        
        # Find the dimension of interest
        index = self.vars.index(var)
        # The numpy .take function takes a slice where dim "index" is value "value".
        ndata = self.data.take(value, index).squeeze()
        nvars = self.vars.copy()
        nvars.remove(var) # Delete the variable from the list
        return factor(ndata, nvars)

    # Implements the sumout operation of a function over a variable
    def sumout(self, var):
        # Handle empty factor
        if self.isEmpty:
            return factor([],[])
        
        # Get dimension number corresponding to variable
        index = self.vars.index(var)
        # the numpy sum operation sums together the restricted slices for every 
        # value of the passed axis
        ndata = self.data.sum(index)
        nvars = self.vars.copy()
        nvars.remove(var) # remove the summed out variable from the list
        
        return factor(ndata, nvars)

    def normalize(self):
        if self.isEmpty:
            return
        # Does this need explaining?
        self.data = self.data/self.data.sum()
        
    # This is a non-critical function it just re-arranges the dimensions
    # so the variables appear in a different order. I implemented it simply 
    # because I once wanted to compare resultsand didn't like manually re-ordering them
    def reorder(self, order):
        if self.isEmpty:
            return
        
        for i, v in enumerate(order):
            curInd = self.vars.index(v)
            self.data = np.swapaxes(self.data, i, curInd) 
            self.vars[curInd] = self.vars[i]
            self.vars[i] = v
    
    # Here I overwrite the multiplication operator so I can do f1*f2 :)
    def __mul__(self, other):
        # Handle cases where one factor or more factors are empty
        if (self.isEmpty and other.isEmpty):
            return factor([],[])
        elif (self.isEmpty):
            return factor(other.data, other.vars)
        elif (other.isEmpty):
            return factor(self.data, self.vars)
        # Multiplication of factors means broadcasting,
        # but we need to be the correct shape first
        
        # Step 1: Align the two factors inputing dummy dimensions when required
            # We need to start by finding the matching components 
            # and reordering indicesSo they line up
        ovars = other.vars.copy()
        odata = other.data.copy()
        
        svars = self.vars.copy()
        sdata = self.data.copy()
        
        # This loop finds when a variable in other is in self,
        # If it finds a match it will move swap the variable order so they match
        # if self does not have the variable in other, a dummy dimension is added
        #  and the variable is added to a copy of self's list
        for i in range(len(ovars)):
            ovar = ovars[i]
            # other and self share a variable
            if (ovar in svars):
                curInd = svars.index(ovar)
                # reorder the variables in self so the the shared variables are in the same dimension
                sdata = np.swapaxes(sdata, i, curInd) 
                svars[curInd] = svars[i]
                svars[i] = ovar
            else:
                # Insert a placeholder dimension in self
                sdata = np.expand_dims(sdata, i)
                svars.insert(i, ovar)
        
        # Finish by adding necessary number of placeholders to the end of other
        for i in range(len(ovars), len(svars)):
            odata = np.expand_dims(odata, i)
        
        # now just multiply the numpy arrays. Broadcasting will ensure matching 
        # dimensions ar multiplied together and placeholder dimensions will be 
        # multiplied by all options
        newData = (sdata*odata).squeeze()
        
        return factor(newData, svars) # return a new factor
    
    # Helper
    def copy(self):
        return factor(self.data.copy(), self.vars.copy())

    # This makes print (f) a little nicer though most times I use f.table() below
    def __str__(self):
        return "Factor with variables: {} \n and Data:\n {}".format(self.vars, self.data)
    
    # Pretty print the truth table of self
    def table(self, p=2):
        if (self.isEmpty):
            return
        
        N = len(self.vars)
        pstr = ""
        for i in range(N):
            pstr += "| {} "
        dstr = pstr + "| {:.{p}f} "
        pstr += "| {} "
        
        temp = pstr.format(*(self.vars + ["P"]))
        print (temp)
        print ("{:->{width}}".format("",width=len(temp)))
        
        for i in range(2**N):
            b = tuple(map(int, "{:0>{width}b}".format(i, width=N)))
            d = list(map(lambda x: "T" if x == 0 else "F", b)) + [self.data[b]]
            print (dstr.format(*d, p=p))
        print ("")
        